In [1]:
import requests, json, urllib.parse

base_url = 'https://itunes.apple.com/search'

artist = 'Yeat'

params = {
    'term': artist,
    'limit': 200,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

In [2]:
import dateutil.parser

# Преобразование строки с датой в объект даты
# **x - скопировать все поля из объекта x
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)

In [3]:
# Убираем лишние поля
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)

# Фильтрация с целью оставить искомого артиста
data = filter(lambda x: artist.lower() in x['artistName'].lower(), data)
data = list(data)

In [4]:
# Функция переводящая миллисекунды в читаемый формат времени
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}H{minutes}M{seconds}S"

In [5]:
from statistics import mean, median
result = {}
result['Artist Name'] = data[0]['artistName']
result['Track Amount'] = len(set(list(map(lambda x: x['trackName'], data))))
result['Popular Genre'] = data[0]['primaryGenreName']
result['Album Amount'] = len(set(list(map(lambda x: x['collectionId'], data))))

result['Total song time'] = convert_millis(sum(set(list(map(lambda x: x['trackTimeMillis'], data)))))
result['Mean song time'] = convert_millis(mean(set(list(map(lambda x: x['trackTimeMillis'], data)))))
result['Median song time'] = convert_millis(median(sorted(set(list(map(lambda x: x['trackTimeMillis'], data))))))
result['Longest name'] = max(data, key=lambda x: x['trackTimeMillis'])['trackName']
result['Longest time'] = convert_millis(max(data, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])
result['Shortest name'] = min(data, key=lambda x: x['trackTimeMillis'])['trackName']
result['Shortest time'] = convert_millis(min(data, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])

result['Total song price'] = sum(list(map(lambda x: x['trackPrice'], data)))
result['Mean song price'] = mean((list(map(lambda x: x['trackPrice'], data))))
result['Median song price'] = median(sorted((list(map(lambda x: x['trackPrice'], data)))))
result['Expensive song name'] = max(data, key=lambda x: x['trackPrice'])['trackName']
result['Most expensive song price'] = max(data, key=lambda x: x['trackPrice'])['trackPrice']
result['Cheapest song name'] = min(data, key=lambda x: x['trackPrice'])['trackName']
result['Cheapest song price'] = min(data, key=lambda x: x['trackPrice'])['trackPrice']

for key, value in result.items():
    print(key + ': ', value)

Artist Name:  Yeat
Track Amount:  127
Popular Genre:  Hip-Hop/Rap
Album Amount:  25
Total song time:  6H1M46S
Mean song time:  0H2M48S
Median song time:  0H2M43S
Longest name:  Lying 4 fun
Longest time:  0H5M3S
Shortest name:  Cmon
Shortest time:  0H1M42S
Total song price:  254.80999999999952
Mean song price:  1.2740500000000001
Median song price:  1.29
Expensive song name:  Monëy so big
Most expensive song price:  1.29
Cheapest song name:  U could tëll
Cheapest song price:  -1.0


In [8]:
from itertools import groupby

album_name = set(list(map(lambda x: x['collectionName'], data)))
album_id = set(list(map(lambda x: x['collectionId'], data)))

print(album_name)
print(album_id)

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']

# Удаление переменной, действие опционально
del sorted_by_album

print(list(album_names.values()))

{'Euphoria Season 2 (An HBO Original Series Soundtrack)', 'Number 1 - Single', 'Wake Up Call', 'Gët Busy - Single', 'Still countin - Single', 'Different Creature', '2 Alivë', 'hollon - Single', 'Trëndi - EP', 'Alivë', '4L', 'Lyfë', 'Rich Minion - Single', 'Up 2 Më', 'Cali - Single', 'Flashey - Single', 'We All We Got - EP', 'Loot (feat. Jban$2Turnt) - Single', 'Talk - Single', '2 Alivë (Geëk Pack)', 'Deep Blue Strips - Single'}
{1642621828, 1643750030, 1612542990, 1578332050, 1645618711, 1567264413, 1616759853, 1473163575, 1425990845, 1602105919, 1607943745, 1583678146, 1638237258, 1612392011, 1576616525, 1576703058, 1436670931, 1645003745, 1448938469, 1609701608, 1616737518, 1469899760, 1629795061, 1583630326, 1634322933}
['Loot (feat. Jban$2Turnt) - Single', 'Deep Blue Strips - Single', 'Wake Up Call', 'Number 1 - Single', 'Different Creature', 'Cali - Single', '4L', 'Alivë', 'Trëndi - EP', 'Gët Busy - Single', 'Up 2 Më', 'Flashey - Single', 'Still countin - Single', '2 Alivë', 'Euph

In [47]:
from statistics import mean, median
result2 = {}

album = album_groups[1643750030]

result2['Album Name'] = album_names[1643750030]
result2['Track Amount'] = len(set(list(map(lambda x: x['trackName'], album))))
result2['Primary Genre'] = list(map(lambda x: x['primaryGenreName'], album))[0]
result2['Album Release'] = album[0]['releaseDate']

result2['Total song time'] = convert_millis(sum(set(list(map(lambda x: x['trackTimeMillis'], album)))))
result2['Mean song time'] = convert_millis(mean(set(list(map(lambda x: x['trackTimeMillis'], album)))))
result2['Median song time'] = convert_millis(int(median(sorted(set(list(map(lambda x: x['trackTimeMillis'], album)))))))
result2['Longest name'] = max(album, key=lambda x: x['trackTimeMillis'])['trackName']
result2['Longest time'] = convert_millis(max(album, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])
result2['Shortest name'] = min(album, key=lambda x: x['trackTimeMillis'])['trackName']
result2['Shortest time'] = convert_millis(min(album, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])

result2['Total song price'] = sum(list(map(lambda x: x['trackPrice'], album)))
result2['Mean song price'] = mean((list(map(lambda x: x['trackPrice'], album))))
result2['Median song price'] = median(sorted((list(map(lambda x: x['trackPrice'], album)))))
result2['Expensive song name'] = max(album, key=lambda x: x['trackPrice'])['trackName']
result2['Most expensive song price'] = max(album, key=lambda x: x['trackPrice'])['trackPrice']
result2['Cheapest song name'] = min(album, key=lambda x: x['trackPrice'])['trackName']
result2['Cheapest song price'] = min(album, key=lambda x: x['trackPrice'])['trackPrice']

for key, value in result2.items():
    print(key + ': ', value)

Album Name:  Lyfë
Track Amount:  12
Primary Genre:  Hip-Hop/Rap
Album Release:  2022-09-09 12:00:00+00:00
Total song time:  0H34M14S
Mean song time:  0H2M51S
Median song time:  0H2M53S
Longest name:  Comë on
Longest time:  0H3M24S
Shortest name:  Wat it feel lykë
Shortest time:  0H2M10S
Total song price:  15.479999999999997
Mean song price:  1.29
Median song price:  1.29
Expensive song name:  Flawlëss (feat. Lil Uzi Vert)
Most expensive song price:  1.29
Cheapest song name:  Flawlëss (feat. Lil Uzi Vert)
Cheapest song price:  1.29
